In [ ]:
import sys
# setting path
sys.path.append('../')

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
HISTORY_PATH = MAIN_PATH + "history/"
MODEL_NAME = "mobilenetv3s_0pe_224ps_0.3theta_0.05sigmaRui_60000nsamples"

history_file = HISTORY_PATH + MODEL_NAME + ".csv"

In [ ]:
df = pd.read_csv(history_file)
df.head()

In [ ]:
def plot_metric(df, metric, title=None):
    plt.plot(df['train_' + metric], label='train')
    plt.plot(df['val_' + metric], label='validation')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    if title is not None:
        plt.title(title)

In [ ]:
plt.figure(figsize=(20, 5))
plt.subplot(1, 3, 1)
plot_metric(df, 'loss', title='Loss')
plt.subplot(1, 3, 2)
plot_metric(df, 'loss_mse', title='Loss MSE')
plt.subplot(1, 3, 3)
plot_metric(df, 'loss_kl', title='Loss KL')
plt.show()